# Corpus compilation
- text type and genres
- characteristics according to task

In [1]:
# Requests in python
import urllib3
# Regular expression library
import re
# Transform html into a tree im memory
from bs4 import BeautifulSoup
import os
# Get just the text in a html document
import justext

In [73]:
# Disable https warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Pass as a browser 
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

# Pool of boots to make requests
http = urllib3.PoolManager(10, headers=user_agent)


class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path # corpus address
        self.max_files = max_files # max amount of sizes
        self.seed_url = seed_url # root url
        self.url_pattern = url_pattern # Select links of interest
        self.visited_links = {} # Hash to store viseted links
        self.to_be_visited = [] # List of link
        
        # If path not exists create it
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
        
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")), "w") as output_file:
            for paragraph in paragraphs:
                # Boilerplate is everthing that is not the main text
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
        links_1 = re.search(self.url_pattern, str(response.data))
        print(links_1)
        links = []
        for group in links_1.groups():
            links.append("https://linuxdicasesuporte.blogspot.com/2019/{}".format(group))
            
        
#         links = [link.get('href') for link in soup.findAll('meta', attrs={'content': re.compile(self.url_pattern)})]
        print(links)
        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

In [74]:
crawler_seguranca = Crawler("data/corpora/seguranca", 50, 
                             "https://linuxdicasesuporte.blogspot.com/2019/04/debian-11-bullseye.html", 
                            '"https://linuxdicasesuporte\.blogspot\.com/(.*?)"')

In [75]:
crawler_seguranca.crawl()

getting page https://linuxdicasesuporte.blogspot.com/2019/04/debian-11-bullseye.html
<re.Match object; span=(864, 925), match='"https://linuxdicasesuporte.blogspot.com/feeds/po>
['https://linuxdicasesuporte.blogspot.com/2019/feeds/posts/default']
getting page https://linuxdicasesuporte.blogspot.com/2019/feeds/posts/default
<re.Match object; span=(858, 919), match='"https://linuxdicasesuporte.blogspot.com/feeds/po>
['https://linuxdicasesuporte.blogspot.com/2019/feeds/posts/default']


IndexError: pop from empty list